# Metaclass basics

Metaclasses allow you to customize the class creation process.

By default, the metaclass for a class is `type`. We can determine what the metaclass of a class is by calling the `type()` function (which, confusingly, is different from the `type` metaclass, even though it has the same name) on the class.

Below, we create a class `MyClass` and we can see that its metaclass is `type`.

In [1]:
class MyClass():
    pass

type(MyClass)

type

We can explicitly set the metaclass of a class with the `metaclass` keyword in the class definition. Below, we set `metaclass=type`. The behavior is the same as the above example since `type` was the default metaclass anyway.

In [2]:
class MyClass(metaclass=type):
    pass

type(MyClass)

type

Typically, if we want to create our own metaclass, it will derive from the `type` metaclass, like so:

In [3]:
class MyMetaClass(type):
    pass

class MyClass(metaclass=MyMetaClass):
    pass

type(MyClass)

__main__.MyMetaClass

When we instantiate a class, the `__call__` method of the class's metaclass is gets called, and it is in charge of calling the class's `__new__` and `__init__` methods to create the new object and initialize it.

In the following, we override `MyMetaClass.__call__` to just print a message and do nothing else. So when we try to instantiate `MyClass`, we never get into `MyClass.__new__` or `MyClass.__init__` methods, and we don't end up creating an instance--`MyClass()` just returns `None`.

Notice that `MyMetaClass.__call__` gets called with `cls=MyClass`. Also, the `args` and `kwargs` are whatever we gave to the `MyClass()` call, which is nothing in this case.

In [4]:
class MyMetaClass(type):
    def __call__(cls, *args, **kwargs):
        print('in MyMetaClass.__call__')
        print(f'  cls: {cls}')
        print(f'  args: {args}')
        print(f'  kwargs: {kwargs}')

class MyClass(metaclass=MyMetaClass):
    def __new__(cls, *args, **kwargs):
        print('in MyClass.__new__')
        return super().__new__(cls, *args, **kwargs)

    def __init__(self, *args, **kwargs):
        print('in MyClass.__init__')

a = MyClass()
print(a)

in MyMetaClass.__call__
  cls: <class '__main__.MyClass'>
  args: ()
  kwargs: {}
None


We can fix our issue by making `MyMetaClass.__call__` call into the default metaclass's `__call__` method with `super().__call__(...)` and return its result.

Now we can see that `MyClass.__new__` and `MyClass.__init__` are getting called, and `MyClass()` returns an instance of `MyClass`.

In [5]:
class MyMetaClass(type):
    def __call__(cls, *args, **kwargs):
        print('in MyMetaClass.__call__')
        return super().__call__(*args, **kwargs)

class MyClass(metaclass=MyMetaClass):
    def __new__(cls, *args, **kwargs):
        print('in MyClass.__new__')
        return super().__new__(cls, *args, **kwargs)

    def __init__(self, *args, **kwargs):
        print('in MyClass.__init__')

a = MyClass()
print(a)

in MyMetaClass.__call__
in MyClass.__new__
in MyClass.__init__


In `MyMetaClass.__call__`, we could also completely avoid the default metaclass's `__call__` method and instead we can explicitly create the new object by calling the `obj = cls.__new__` method and call `obj.__init__` to initialize it.

In [6]:
class MyMetaClass(type):
    def __call__(cls, *args, **kwargs):
        print('in MyMetaClass.__call__')
        
        # This is basically what `super().__call__(...)` would do
        obj = cls.__new__(cls, *args, **kwargs)
        obj.__init__(*args, **kwargs)
        return obj

class MyClass(metaclass=MyMetaClass):
    def __new__(cls, *args, **kwargs):
        print('in MyClass.__new__')
        return super().__new__(cls, *args, **kwargs)

    def __init__(self, *args, **kwargs):
        print('in MyClass.__init__')

a = MyClass()
print(a)

in MyMetaClass.__call__
in MyClass.__new__
in MyClass.__init__


## Example: Singleton pattern

An example of when you might want to use a custom metaclass is the singleton pattern. Below is a fairly typical implementation of a singleton class without using a custom metaclass.


In [7]:
class MyClassSingleton():
    _instance = None
    
    def __new__(cls, *args, **kwargs):
        print('in MyClassSingleton.__new__')
        if cls._instance is None:
            cls._instance = super().__new__(cls, *args, **kwargs)

        return cls._instance
    
    def __init__(self, *args, **kwargs):
        print('in MyClassSingleton.__init__')
        
a = MyClassSingleton()
b = MyClassSingleton()
assert a is b
print(a)

in MyClassSingleton.__new__
in MyClassSingleton.__init__
in MyClassSingleton.__new__
in MyClassSingleton.__init__



`MyClassSingleton()` successfully returns the same instance each time it is called. However, there are a few issues with this.

The most obvious issue is that `__init__` gets called each time. It's possible that you might want that behavior, but you'd probably have to add some logic to `__init__` that detects whether it was called before so that you avoid resetting any properties on the object that you don't want to change.

Another problem is that the singleton pattern is tightly coupled with the class. So if we wanted to apply the pattern to another class, we'd have to copy some boilerplate code. Furthermore, we don't have any concept of a regular non-singleton `MyClass`.

It would be nice if we could just implement `MyClass` normally, and then just tack on something that says, "hey by the way, this is a singleton". To do this, we can make a metaclass that implements the singleton pattern.

In [8]:
class Singleton(type):
    _instance = None
    
    def __call__(cls, *args, **kwargs):
        print('in Singleton.__call__')
        if cls._instance is None:
            cls._instance = super().__call__(*args, **kwargs)
        return cls._instance

class MyClass():
    def __new__(cls, *args, **kwargs):
        print('in MyClass.__new__')
        return super().__new__(cls, *args, **kwargs)
    
    def __init__(self, *args, **kwargs):
        print('in MyClass.__init__')

class MyClassSingleton(MyClass, metaclass=Singleton):
    pass
        
a = MyClassSingleton()
b = MyClassSingleton()
assert a is b
print(a)

c = MyClass()
assert c is not a
print(c)

in Singleton.__call__
in MyClass.__new__
in MyClass.__init__
in Singleton.__call__
in MyClass.__new__
in MyClass.__init__


Now we avoid calling `MyClass.__new__` and `MyClass.__init__` multiple times if `MyClassSingleton()` is called more than once, and we can create separate non-singleton `MyClass` objects.

We can also apply this `Singleton` metaclass to other classes, shown below.

In [9]:
class MyOtherClassSingleton(metaclass=Singleton):
    def __new__(cls, *args, **kwargs):
        print('in MyClass.__new__')
        return super().__new__(cls, *args, **kwargs)
    
    def __init__(self, *args, **kwargs):
        print('in MyClass.__init__')

d = MyOtherClassSingleton()
e = MyOtherClassSingleton()

assert d is e
assert d is not a
print(d)

in Singleton.__call__
in MyClass.__new__
in MyClass.__init__
in Singleton.__call__


Notice that in this case, we're not creating a non-singleton version of `MyOtherClassSingleton`, but it would be easy to do so, just like we did with `MyClass` and `MyClassSingleton` before.

So implementing the singleton pattern as a metaclass gives us some nice flexibility.

## Example: 

In [10]:
import gc

class MyMetaClass(type):
    def __call__(cls, *args, **kwargs):
        print('in MyMetaClass.__call__')
        a = super().__call__(*args, **kwargs)
        a.__del__ = MyMetaClass._subclass_del
        print(f'  {a}')
        return a
    
    def __del__(cls):
        print('in MyMetaClass.__del__')
        print(cls)
    
    def _subclass_del(self):
        print('in _subclass_del')

class MyClass(metaclass=MyMetaClass):
    def __repr__(self):
        return 'MyClass'
    
    def __del__(self):
        print('in MyClass.__del__')
        
a = MyClass()
del a
gc.collect()

in MyMetaClass.__call__
  MyClass
in MyClass.__del__


44